In [186]:
import re
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from datasets import load_dataset
import numpy as np
import pandas as pd

In [187]:
dataset = load_dataset('craigslist_bargains', split= 'train')
test_dataset = load_dataset('craigslist_bargains', split= 'validation')

Using custom data configuration default
Reusing dataset craigslist_bargains (/Users/kabirbatra/.cache/huggingface/datasets/craigslist_bargains/default/1.1.0/9e4e64d13ea36e435a8f56f9305d3c771115909b7fe83269bf7cd8dd40302338)
Using custom data configuration default
Reusing dataset craigslist_bargains (/Users/kabirbatra/.cache/huggingface/datasets/craigslist_bargains/default/1.1.0/9e4e64d13ea36e435a8f56f9305d3c771115909b7fe83269bf7cd8dd40302338)


In [188]:
df = pd.DataFrame(dataset)
df.rename(columns={'dialogue_acts':'intent','utterance':'convo'}, inplace=True)
test_df = pd.DataFrame(test_dataset)

In [189]:
# def notWaste(value):
#     if value not in ['intent','price']: return True
#     else: return False
 
def intentExtraction(value):
    intent = re.findall('[a-zA-Z\-]{2,}', str(value))
    intent = list(filter(lambda x: x!='intent' and x!='price', intent))
    return intent 

def priceExtraction(value):
    price = re.findall('[\d\.\-]{2,}', str(value))
    return price

In [190]:
pd.set_option('display.max_colwidth', 1)
intent_matrix = df['intent'].apply(lambda x: pd.Series(intentExtraction(x)))
intent_matrix = pd.DataFrame(intent_matrix)

price_matrix = df['intent'].apply(lambda x: pd.Series(priceExtraction(x)))
price_matrix = pd.DataFrame(price_matrix)

price_matrix

<ipython-input-190-ce2cd72b4437>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  intent_matrix = df['intent'].apply(lambda x: pd.Series(intentExtraction(x)))


,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
0,5.0,-1.0,5.0,8.0,6.0,7.0,7.0,7.0,-1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1.0,-1.0,-1.0,-1.0,100.0,-1.0,120.0,150.0,145.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-1.0,-1.0,-1.0,-1.0,400.0,480.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-1.0,-1.0,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5242,-1.0,-1.0,-1.0,-1.0,5600.0,6000.0,-1.0,-1.0,6000.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5243,-1.0,-1.0,175.0,-1.0,175.0,250.0,-1.0,250.0,-1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5244,-1.0,-1.0,-1.0,-1.0,100.0,150.0,125.0,130.0,-1.0,130.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5245,-1.0,-1.0,-1.0,-1.0,35.0,35.0,-1.0,-1.0,-1.0,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [191]:
print(intent_matrix.shape == price_matrix.shape)

True


In [192]:
intent_matrix.replace({ np.nan: -1.0, 'accept': -1.0}, inplace=True)
price_matrix.replace(np.nan, -1.0, inplace=True)

## 'offer' is 'final offer'
intent_matrix.replace({'init-price':1, 'insist':2, 'counter-price':3, 'agree':4,
       'intro':5, 'inquiry':6, 'disagree':7, 'vague-price':8, 'inform':9, 'unknown':10, 'offer': 11}, inplace=True)


In [193]:
intent_matrix

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
0,1.0,10.0,2,3.0,3,3,4,11,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,5.0,10.0,6,7.0,1,8,3,3,3,4,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,5.0,10.0,6,9.0,1,3,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5242,5.0,10.0,6,6.0,1,3,6,9,11,10,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5243,5.0,10.0,1,7.0,2,3,10,11,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5244,5.0,7.0,6,9.0,1,3,3,3,10,11,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5245,5.0,10.0,6,7.0,1,4,3,10,10,11,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [194]:
price_matrix

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
0,5.0,-1.0,5.0,8.0,6.0,7.0,7.0,7.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,-1.0,-1.0,-1.0,-1.0,100.0,-1.0,120.0,150.0,145.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,-1.0,-1.0,-1.0,-1.0,400.0,480.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5242,-1.0,-1.0,-1.0,-1.0,5600.0,6000.0,-1.0,-1.0,6000.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5243,-1.0,-1.0,175.0,-1.0,175.0,250.0,-1.0,250.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5244,-1.0,-1.0,-1.0,-1.0,100.0,150.0,125.0,130.0,-1.0,130.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5245,-1.0,-1.0,-1.0,-1.0,35.0,35.0,-1.0,-1.0,-1.0,40.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
